In [1]:
from pathlib import Path
from dataclasses import dataclass 
import requests
import numpy as np
import torch
import torch.nn as nn
from jaxtyping import Float, Int

import torch.optim as optim
import torch.nn.functional as F
from typing import List
import requests

import unicodedata
import json
from collections import Counter, defaultdict
import base64
from IPython.display import Image, display


@dataclass 
class Config:
    d_model : int 
    d_vocab : int
    d_hidden : int # for MLP
    n_layers : int
    # d_head : int # for Attn (if separate wq and wk)
    #no n_context
    #name var : type

# guttenburg dataset code in existing notebooks

### Dimensions

\begin{align*}
    &\text{d-model} = d_m & : & \text{model dimension (num neurons)} \\
    &d_v = \text{d-vocab} & : & \text{vocab dimension} \\
    &n_c = \text{n-context} & : & \text{context window (len of seq entered)}
\end{align*}

Where $d_n << d_m$

## Getting Data

In [3]:
import requests

url = "https://www.gutenberg.org/files/67098/67098-0.txt"
response = requests.get(url)
text = response.text

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

start_idx = text.find(start_marker)
end_idx = text.find(end_marker)
content_area = text[start_idx:end_idx].split("\n", 1)[1]

chapter_idx = content_area.upper().find("CHAPTER I")

raw_text = content_area[chapter_idx:]
print(raw_text[:500000])


CHAPTER I

                      IN WHICH WE ARE INTRODUCED TO
                     WINNIE-THE-POOH AND SOME BEES,
                          AND THE STORIES BEGIN


Here is Edward Bear, coming downstairs now, bump, bump, bump, on the
back of his head, behind Christopher Robin. It is, as far as he knows,
the only way of coming downstairs, but sometimes he feels that there
really is another way, if only he could stop bumping for a moment and
think of it. And then he feels that perhaps there isn't. Anyhow, here he
is at the bottom, and ready to be introduced to you. Winnie-the-Pooh.

When I first heard his name, I said, just as you are going to say, "But
I thought he was a boy?"

"So did I," said Christopher Robin.

"Then you can't call him Winnie?"

"I don't."

"But you said----"

"He's Winnie-ther-Pooh. Don't you know what '_ther_' means?"

"Ah, yes, now I do," I said quickly; and I hope you do too, because it
is all the explanation you are going to get.

Sometimes Winnie-the-Pooh likes

## Tokenize Text

In [ ]:
class Tokenizer:
    def __init__(self, text):
        # clean and sort data 
        cleaned = self.clean_text(text)
        self.chars = sorted(set(cleaned))
        # vocab size 
        self.vocab_size = len(self.chars)

        # vocab map 
        self.encode = {}
        self.decode = {}

        for i, chars in enumerate(self.chars):
            self.encode[chars] = i
            self.decode[i] = chars

        # sanity check 
        print(f"Vocab chars:" ,self.chars)
        print(f"Vocab size:" , self.vocab_size)

    #simplify
    #all letters lowercase
    #each punctuation into a token each letter a token
    #get a set of tokens
    #this set is d_vocab

    # clean the text 
    def clean_text(self, text: str) -> list[str]:
        return [x for x in text.lower() if x.isalpha() or x in " .!?"]

    # encoder 
    def tokenize(self, text):
    # will update words to nums with vocab map 
        cleaned = self.clean_text(text)

        tokens = []
        
        for char in cleaned:
            if char in self.encode:
                tokens.append(self.encode[char])

        print("Encoded tokens:", tokens)
        return tokens

    # decoder 
    def detokenize(self, tokens):
    #inverse of tokenize (nums to words )
        words = []
        for id in tokens:
            if id in self.decode:
                words.append(self.decode[id])

        final = "".join(words)
        print("Decoded text:", final)
        
        return final
        

In [18]:
# test case for tokenizer class 
tokenizer = Tokenizer(raw_text)

sample_text = "Chapter I. The starting of the journey!"

encoded = tokenizer.tokenize(sample_text)
decoded = tokenizer.detokenize(encoded)

Vocab chars: [' ', '!', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ']
Vocab size: 31
Encoded tokens: [6, 11, 4, 19, 23, 8, 21, 0, 12, 2, 0, 23, 11, 8, 0, 22, 23, 4, 21, 23, 12, 17, 10, 0, 18, 9, 0, 23, 11, 8, 0, 13, 18, 24, 21, 17, 8, 28, 1]
Decoded text: chapter i. the starting of the journey!


## Multilayer Perceptron
$$
    \texttt{MLP}(\mathbf{X}) = W_d \cdot \sigma_{\texttt{ReLU}} (W_u \cdot x + b_u) + b_d, \qquad \texttt{MLP} : \mathbb{R}^{d_m} \to \mathbb{R}^{d_m}
$$

In [ ]:
class MLP(nn.Module):
    def __init__(self, config: Config): # matrices to initialize
        super().__init__()
        self.linear_up: nn.Linear = nn.Linear(config.d_model, config.d_hidden)
        self.linear_down: nn.Linear = nn.Linear(config.d_hidden, config.d_model)
    
    def forward(self, x: Float[torch.Tensor, "* d_model"]) -> Float[torch.Tensor, "* d_model"]:
        x = self.linear_up(x)
        x = torch.relu(x)
        x = self.linear_down(x)
        return x  

## Attention Head
### Weight Matrix
$$
    \mathbf{W}_{QK} := \mathbf{W}_{Q} \cdot \mathbf{W}_{K}^T, \qquad \mathbf{W}_{Q}, \mathbf{W}_{K} \in \mathbb{R}^{d_m \times d_n}
$$

### Autoregressive Masking (M) Matrix
$$
    M_{i,j} = 
    \begin{cases}
        0 &j \geq i \\
        -\infty &j < i>
    \end{cases}
$$

### Forward Pass
$$A(\mathbf{X}) = \sigma_{\text{softmax}} (\mathbf{X} \; \mathbf{W}_{QK} \; \mathbf{X}^\text{T} + \mathbf{M}) \; \mathbf{X} \; \mathbf{W}_{OV}$$

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        # weights (use nn.parameter) to create a matrix to track gradients
        self.wqk = nn.Parameter(torch.randn(config.d_model, config.d_model))
        self.wov = nn.Parameter(torch.randn(config.d_model, config.d_model))

        ## Create M Matrix
    def M_matrix(n):
        # matrix with 0 at and below the diagonal and -inf above the diagonal
        M = torch.ones((n, n))
        M = torch.triu(M, diagonal=1)
        M = M.masked_fill(M == 1, float('-inf'))
        print(M)
        
    
    def forward(self, x: Float[torch.Tensor, "* d_model"]) -> Float[torch.Tensor, "* d_model"]:
        # use weights to compute A⨉
        # X as input: n_seq by d_model
        n_seq = x.shape[0]
        M = self.M_matrix(n_seq)
        attention_pattern = x @ self.wqk @ x.T + M
        attention_of_X = nn.softmax(attention_pattern) @ x @ self.wov
        
        return attention_of_X

### Transformer Block
$$
    \text{TB}(X) = X + A(X) + \text{MLP}(X), \qquad \text{TB}: \mathbb{R}^{n_c \times d_m} \to \mathbb{R}^{n_c \times d_m}
$$

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        #self.ln = nn.LayerNorm(config)
        self.mlp = MLP(config)
        self.attention = AttentionHead(config)


    def foward(self, x: Float[torch.Tensor, "* d_model"]) -> Float[torch.Tensor, "* d_model"]:
        #output = x + mlp(x) + attentionhead(x)   
        output_x = x + self.mlp(x) + self.attention(x)
        #x = self.ln(x_1)
        #x = self.ln(x_2)

        return(output_x)

In [ ]:
class transformer(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        self.token_embedding = nn.Embedding(config.d_vocab, config.d_model)
        #self.transformerblocks = nn.modules list of transformer blocks
        self.transformerblocks = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_layers)])
        
    def forward(self, x: Int[torch.Tensor, "n_context"]) -> Float[torch.Tensor, "n_context d_vocab"]:
        x = self.token_embedding(x) # converts int d-vector to d-model vector
        for i in range(n_layers):
            x = self.transformerblocks[i](x)
        x = x @ self.token_embedding.weight.T # unembedding 
        #n_contex long - sequence if ints of length n  - float tneosry by n_model  and output is float tencsosr by d-vocab \n",
        #d_model to d_vocab transpose or do a lineear map  - unembed nn.linear
        #dmodel to dvocab 

        return x
        
        
        
# use nn.ModuleList for TB seqeunce & MHA (to create a list of TBS)
# print(f"{x.shape = }") for debugging

# pick a unique dataset to train data on

# if traning models: aim for < 10 million parameters for now
#   sum(x.numel() for x in mymodel.parameters())
